# 278k Emotion Labeled Spotify Songs  
https://www.kaggle.com/datasets/abdullahorzan/moodify-dataset  
- Labels: {'sad': 0, 'happy': 1, 'energetic': 2, 'calm': 3}  


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. Import Modules

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC
from sklearn import set_config



# 2. Load Dataset

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Epita/project/DataProject/dataset/new_278k_labelled_uri.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207929 entries, 0 to 207928
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        207929 non-null  int64  
 1   danceability      207929 non-null  float64
 2   energy            207929 non-null  float64
 3   loudness          207929 non-null  float64
 4   speechiness       207929 non-null  float64
 5   acousticness      207929 non-null  float64
 6   instrumentalness  207929 non-null  float64
 7   liveness          207929 non-null  float64
 8   valence           207929 non-null  float64
 9   tempo             207929 non-null  float64
 10  labels            207929 non-null  int64  
dtypes: float64(9), int64(2)
memory usage: 17.5 MB


In [ ]:
df.isnull().sum()

Unnamed: 0          0
danceability        0
energy              0
loudness            0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
labels              0
dtype: int64

In [ ]:
df.drop(['Unnamed: 0'], axis = 1, inplace = True)
df.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,labels
0,0.638,0.781,-6.848,0.0285,0.0118,0.009530,0.3490,0.250,122.985,1
1,0.560,0.810,-8.029,0.0872,0.0071,0.000008,0.2410,0.247,170.044,1
2,0.525,0.699,-4.571,0.0353,0.0178,0.000088,0.0888,0.199,92.011,0
3,0.367,0.771,-5.863,0.1060,0.3650,0.000001,0.0965,0.163,115.917,1
4,0.572,0.837,-7.876,0.0367,0.0197,0.000000,0.1630,0.627,100.343,1


# 3. Data Preprocessing (knn_ver1에서 전처리한 데이터셋 사용해서 생략함)

# 4. Split Train/Test set

In [ ]:
X = df.iloc[:, :-1] ## duration ~ spec_rate
y = df.iloc[:, -1] ## labels

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=42)

X_train.shape, y_train.shape

((145550, 9), (145550,))

# 5.Define & Fit the model

In [ ]:
# ref = https://www.kaggle.com/code/raheeldibaji/accuracy-96-6

tf2_lr = ColumnTransformer([('Scaler', StandardScaler(), slice(0,9))]) ##
svm = SVC(kernel='linear')
pipeline = Pipeline([('Scaler',tf2_lr),('Support Vector Machine', svm)])

pipeline.fit(X_train, y_train)


# Calculate the accuracy of the classifier
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

set_config(display='diagram')
pipeline

Accuracy: 0.8332772247070328


Pipeline(steps=[('Scaler',
                 ColumnTransformer(transformers=[('Scaler', StandardScaler(),
                                                  slice(0, 9, None))])),
                ('Support Vector Machine', SVC(kernel='linear'))])

In [ ]:
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold
from sklearn.metrics import accuracy_score

tf2_lr = ColumnTransformer([('Scaler', MinMaxScaler(), slice(0, 9))])
svm = SVC(kernel='linear')
pipeline = Pipeline([('Scaler', tf2_lr), ('Support Vector Machine', svm)])

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Perform cross-validation and get predicted values for each fold
y_pred_cv = cross_val_predict(pipeline, X_train, y_train, cv=cv)

# Fit and evaluate the pipeline using cross-validation
accuracies = cross_val_score(pipeline, X_train, y_train, cv=cv)

# Calculate the mean and standard deviation of the cross-validation results
mean_accuracy = accuracies.mean()
std_accuracy = accuracies.std()

print("Mean Accuracy:", mean_accuracy)
print("Standard Deviation:", std_accuracy)

set_config(display='diagram')
pipeline

Mean Accuracy: 0.8343180949960408
Standard Deviation: 0.0014232905322349404


Pipeline(steps=[('Scaler',
                 ColumnTransformer(transformers=[('Scaler', MinMaxScaler(),
                                                  slice(0, 9, None))])),
                ('Support Vector Machine', SVC(kernel='linear'))])

In [ ]:
y_pred_cv, len(y_pred_cv)

(array([1, 3, 3, ..., 1, 0, 1]), 145550)

In [ ]:
y_test

154559    0
85146     3
177403    3
75231     0
99695     3
         ..
32867     0
175042    1
180330    2
63363     2
91883     0
Name: labels, Length: 62379, dtype: int64

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
f1_score_cv = f1_score(y_train, y_pred_cv, average='weighted')


print("Cross-Validated F1-Score:", f1_score_cv)


Cross-Validated F1-Score: 0.8341504931969845


In [ ]:
macro_score = f1_score(y_train, y_pred_cv, average='macro')
print('macro =', macro_score)

macro = 0.8417062164867213
